In [ ]:
#hello

In [1]:
from transformers import CLIPVisionConfig, CLIPVisionModel, AutoProcessor, CLIPForImageClassification
import torch

/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
configuration = CLIPVisionConfig()
model.config


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPVisionConfig {
  "_attn_implementation_autoset": true,
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 768,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 32,
  "projection_dim": 512,
  "torch_dtype": "float32",
  "transformers_version": "4.50.0"
}

In [7]:
for name, param in model.named_parameters():
    print(name, param.shape)


vision_model.embeddings.class_embedding torch.Size([768])
vision_model.embeddings.patch_embedding.weight torch.Size([768, 3, 32, 32])
vision_model.embeddings.position_embedding.weight torch.Size([50, 768])
vision_model.pre_layrnorm.weight torch.Size([768])
vision_model.pre_layrnorm.bias torch.Size([768])
vision_model.encoder.layers.0.self_attn.k_proj.weight torch.Size([768, 768])
vision_model.encoder.layers.0.self_attn.k_proj.bias torch.Size([768])
vision_model.encoder.layers.0.self_attn.v_proj.weight torch.Size([768, 768])
vision_model.encoder.layers.0.self_attn.v_proj.bias torch.Size([768])
vision_model.encoder.layers.0.self_attn.q_proj.weight torch.Size([768, 768])
vision_model.encoder.layers.0.self_attn.q_proj.bias torch.Size([768])
vision_model.encoder.layers.0.self_attn.out_proj.weight torch.Size([768, 768])
vision_model.encoder.layers.0.self_attn.out_proj.bias torch.Size([768])
vision_model.encoder.layers.0.layer_norm1.weight torch.Size([768])
vision_model.encoder.layers.0.layer

In [9]:
from PIL import Image
import requests
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [10]:
inputs = processor(images=image, return_tensors="pt")

In [29]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [26]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [30]:
outputs = model(inputs['pixel_values'])

In [31]:
type(outputs)

transformers.modeling_outputs.BaseModelOutputWithPooling

In [32]:
type(outputs.last_hidden_state)

torch.Tensor

In [33]:
outputs.last_hidden_state.shape

torch.Size([1, 50, 768])

In [35]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [37]:
classifier = CLIPForImageClassification.from_pretrained("openai/clip-vit-base-patch32")


Some weights of CLIPForImageClassification were not initialized from the model checkpoint at openai/clip-vit-base-patch32 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
classifier = CLIPForImageClassification.from_pretrained("openai/clip-vit-base-patch32", num_labels=2)

Some weights of CLIPForImageClassification were not initialized from the model checkpoint at openai/clip-vit-base-patch32 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
classifier.config

CLIPConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "attention_dropout": 0.0,
    "bos_token_id": 0,
    "dropout": 0.0,
    "eos_token_id": 2,
    "hidden_act": "quick_gelu",
    "hidden_size": 512,
    "initializer_factor": 1.0,
    "initializer_range": 0.02,
    "intermediate_size": 2048,
    "layer_norm_eps": 1e-05,
    "max_position_embeddings": 77,
    "model_type": "clip_text_model",
    "num_attention_heads": 8,
    "num_hidden_layers": 12,
    "projection_dim": 512,
    "torch_dtype": "float32",
    "vocab_size": 49408
  },
  "torch_dtype": "float32",
  "transformers_version": "4.50.0",
  "vision_config": {
    "attention_dropout": 0.0,
    "dropout": 0.0,
    "hidden_act": "quick_gelu",
    "hidden_size": 768,
    "image_size": 224,
    "initializer_factor": 1.0,
    "initializer_range": 0.02,
    "

In [41]:
with torch.no_grad(): 
    logits = classifier(**inputs).logits

In [42]:
logits

tensor([[-0.3076, -0.3316]])

In [43]:
classifier.parameters()

<generator object Module.parameters at 0x7fe3c4227300>

In [ ]:
import lightning as L

class CLIPImageClassifier(L.LightningModule):
    def __init__(self, num_labels=2):
        super().__init__()
        self.model = CLIPForImageClassification.from_pretrained("openai/clip-vit-base-patch32", num_labels=num_labels)
        
    def forward(self, pixel_values):
        return self.model(pixel_values).logits
    
    def training_step(self, batch, batch_idx):
        pixel_values = batch['pixel_values']
        labels = batch['labels']
        logits = self(pixel_values)
        loss = self.model.compute_loss(logits, labels)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)